In [1]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from instascrape import * 
import plotly.express as px
from tqdm import tqdm
from time import sleep
import os

In [2]:
def get_post_links(driver):
    ##read posts
    images = driver.find_elements_by_tag_name('a')
    images = [image.get_attribute('href') for image in images]
    matching = [s for s in images if "https://www.instagram.com/p/" in s] #todos los post tienen esta forma
    return matching

C:\Users\milaronix\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
#Ubicacion del driver de Chrome
file_path = 'C:/python_webdriver/chromedriver.exe'

#Inicializar el driver de Chrome (El explorador Chrome debe estar instalado en el equipo)
driver = webdriver.Chrome(file_path)

#Le decimos la URL a que queremos ir
driver.get("http://www.instagram.com")

#Obtenemos los elementos donde la página espera recibir el usuario y la contraseña
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

#Enviamos el usuario y contraseña de nuestra cuenta de Instragram para poder ingresar
username.clear()
username.send_keys(os.getenv('millie_ig_user'))
password.clear()
password.send_keys(os.getenv('millie_ig_password'))

#Hacemos click en el boton [Iniciar sesión]
Login_button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

#Hacemos click en el primer texto de [Ahora no]
not_now = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Ahora no')]"))).click()

#Hacemos click en el segundo texto de [Ahora no]
not_now2 = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Ahora no')]"))).click()

C:\Users\milaronix\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
##Ubicamos el objeto de la caja de búsqueda
searchbox = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Buscar']")))
searchbox.clear()

#Definimos y enviamos el keyword (usuario) que queremos buscar y esperamos dos segundos
keyword = "dechuchos"
searchbox.send_keys(keyword)
time.sleep(2)

#Presionamos la tecla Enter dos veces esperando dos segundos despues de cada uno
searchbox.send_keys(Keys.ENTER)
time.sleep(2)
searchbox.send_keys(Keys.ENTER)
time.sleep(2)

C:\Users\milaronix\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
# Definimos cuanto queremos escribir entre cada scroll
SCROLL_PAUSE_TIME = 3

# Obtenemos la altura inicial del scroll
last_height = driver.execute_script("return document.body.scrollHeight")
print('initial ',last_height)

# Aca guardaremos los links. Inicializamos con los primeros que vemos al ingresar al perfil
acum = get_post_links(driver)
while True:
    # Hacemos scroll hasta abajo
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Esperamos a que cargue la pagina
    time.sleep(SCROLL_PAUSE_TIME)

    # Agregamos los links a los que tenemos acceso en este momento. Puede incluir repetidos
    acum.extend(get_post_links(driver))
    print('acumulados ',len(acum))    

    # Calculamos la nueva altura para scroll y compramos con la anterior para saber si ya llegamos al final
    new_height = driver.execute_script("return document.body.scrollHeight")
    print('new ',last_height)

    # Validamos si ya llegamos hasta el final
    if new_height == last_height:
        break
    last_height = new_height

C:\Users\milaronix\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
initial  2171
acumulados  36
new  2171
acumulados  72
new  3455
acumulados  114
new  4739
acumulados  156
new  6023
acumulados  198
new  7307
acumulados  240
new  8591
acumulados  282
new  9875
acumulados  324
new  11159
acumulados  366
new  12443
acumulados  408
new  13727
acumulados  450
new  15011
acumulados  492
new  16295
acumulados  534
new  17579
acumulados  576
new  18863
acumulados  618
new  20147
acumulados  660
new  21431
acumulados  702
new  22715
acumulados  744
new  23999
acumulados  786
new  25283
acumulados  828
new  26567

In [6]:
# Como tenemos duplicados al crear un diccionario volvemos unicos los registros
posts_links = list(dict.fromkeys(acum))
print ("Cantidad de links de Posts: ", len(posts_links))


Cantidad de links de Posts:  933
C:\Users\milaronix\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
# Inicializamos un diccionario vacío donde guardaremos la información de los posts
acum = []

# Como este proceso es lento agregamos una barra para saber el porcentaje de avance
pbar = tqdm(posts_links)

# Iteramos sobre los links 
for char in pbar:
    pbar.set_description("Processing %s" % char)
    post = Post(char) # Creamos el objeto post de insta-scrape con el link de la iteración
    try:
        post.scrape() # Pedimos los datos del post
    except Exception:
        pass
    acum.append(post.to_dict()) #Diccionario donde acumulamos los dataframes que devulve insta-scrape
    time.sleep(5) # Esta pausa de 5 segundos es para que Instagram no nos bloquee por abuso

# Convertimos a un dataframe el diccionario donde acumulamos la data
posts_df = pd.DataFrame(acum)

C:\Users\milaronix\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Processing https://www.instagram.com/p/BWV4bJwhcVK/: 100%|██████████| 933/933 [1:27:45<00:00,  5.64s/it]


In [8]:
fig = px.scatter(x=posts_df["upload_date"], y=posts_df["likes"])
fig.show()

C:\Users\milaronix\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
# Exportar los datos a un archivo .txt
posts_df.to_csv('dechuchos.txt',sep='|')

C:\Users\milaronix\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

